# Forecasting Pipeline - Model Training and Evaluation

This notebook builds and evaluates machine learning models to predict Europe Base Port container prices 1-week ahead.

---

## Approach

1. **Load prepared data** from `data/processed/model_data.csv`
2. **Exclude current price** from features (prevent data leakage)
3. **Train-test split** using time-based split (80/20)
4. **Baseline models** using lagged prices for comparison
5. **Advanced models** with hyperparameter tuning
6. **Model evaluation** using RMSE, MAE, MAPE, R²
7. **Feature importance analysis**
8. **Final model selection and validation**

## Models to Test

- **Baseline**: Naive (last week's price), Moving Average (4-week)
- **Linear Models**: Linear Regression, Ridge, Lasso
- **Tree-based**: Random Forest, XGBoost, Gradient Boosting


In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Machine learning libraries
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

Libraries imported successfully!


## Step 1: Load Prepared Model Data

In [2]:
# Load the prepared model data from the previous notebook
df_model = pd.read_csv('data/processed/model_data.csv', parse_dates=['Date'], index_col='Date')

print(f"Loaded model data: {df_model.shape[0]} rows, {df_model.shape[1]} columns")
print(f"Date range: {df_model.index.min().date()} to {df_model.index.max().date()}")
print(f"\nColumns: {list(df_model.columns)}")
print(f"\nMissing values:\n{df_model.isnull().sum()[df_model.isnull().sum() > 0]}")
print(f"\nFirst 5 rows:")
df_model.head()

Loaded model data: 384 rows, 288 columns
Date range: 2018-01-05 to 2025-08-15

Columns: ['SCFI_Index', 'Europe_Base_Price', 'global_total_events', 'global_disruption_events', 'extreme_crisis_events', 'high_velocity_media_events', 'black_swan_candidate_events', 'global_avg_impact', 'global_worst_event_impact', 'global_avg_sentiment', 'global_total_media_mentions', 'global_peak_event_media', 'maritime_conflict_events', 'infrastructure_attack_events', 'trade_restriction_events', 'protest_events', 'middle_east_disruption', 'asia_disruption', 'europe_disruption', 'russia_ukraine_disruption', 'egypt_disruption', 'yemen_disruption', 'unique_sources', 'Brent_Price', 'sh_portcalls_container', 'sh_portcalls_dry_bulk', 'sh_portcalls_general_cargo', 'sh_portcalls_roro', 'sh_portcalls_tanker', 'sh_portcalls_cargo', 'sh_portcalls', 'sh_import_container', 'sh_import_dry_bulk', 'sh_import_general_cargo', 'sh_import_roro', 'sh_import_tanker', 'sh_import_cargo', 'sh_import', 'sh_export_container', 'sh_e

,SCFI_Index,Europe_Base_Price,global_total_events,global_disruption_events,extreme_crisis_events,high_velocity_media_events,black_swan_candidate_events,global_avg_impact,global_worst_event_impact,global_avg_sentiment,...,choke_malacca_strait_capacity_lag_2w_lag_2w,choke_suez_canal_capacity_lag_1w_lag_1w,choke_suez_canal_capacity_lag_1w_lag_2w,choke_suez_canal_capacity_lag_2w_lag_1w,choke_suez_canal_capacity_lag_2w_lag_2w,choke_taiwan_strait_capacity_lag_1w_lag_1w,choke_taiwan_strait_capacity_lag_1w_lag_2w,choke_taiwan_strait_capacity_lag_2w_lag_1w,choke_taiwan_strait_capacity_lag_2w_lag_2w,trade_restriction_events_lag_1w_lag_1w
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-05,816.58,888,251442,70842,19044,53,8,0.533128,-10,-2.197484,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-12,839.72,897,296571,81076,22254,27,5,0.576733,-10,-2.007298,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-19,840.36,891,312641,84236,25340,48,5,0.582549,-10,-1.900899,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17003.0
2018-01-26,858.60,907,304329,80164,22848,30,7,0.634546,-10,-1.806326,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18105.0
2018-02-02,883.59,912,300174,81431,24080,97,5,0.560202,-10,-1.974838,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18214.0


In [3]:
# ====================================================================
# VERIFICATION STEP: Check for new features
# ====================================================================
print("="*70)
print("VERIFYING LOADED FEATURES")
print("="*70)

# Check for some of the new advanced features we created
new_feature_samples = [
    'Europe_Base_Price_roll_mean_4w_lag_1w',
    'Europe_Base_Price_pct_change_1w_lag_1w',
    'price_to_brent_ratio_lag_1w',
    'WTI_Price_lag_8w'
]

found_features = [f for f in new_feature_samples if f in df_model.columns]

print(f"Total columns loaded: {len(df_model.columns)}")

if len(found_features) > 0:
    print("\n✅ SUCCESS: New advanced features are present!")
    print(f"   Found {len(found_features)} out of {len(new_feature_samples)} sample features.")
    print(f"   Sample: {found_features}")
else:
    print("\n❌ FAILURE: New features are MISSING.")
    print("   This confirms that 'model_data.csv' is outdated.")
    print("\n   SOLUTION: Please go back to '02_data_understanding.ipynb' and run the entire notebook ('Run All').")
    print("   Then, come back here and re-run this entire forecasting pipeline.")
    
print("="*70)


VERIFYING LOADED FEATURES
Total columns loaded: 288

❌ FAILURE: New features are MISSING.
   This confirms that 'model_data.csv' is outdated.

   SOLUTION: Please go back to '02_data_understanding.ipynb' and run the entire notebook ('Run All').
   Then, come back here and re-run this entire forecasting pipeline.


## Step 2: Prepare Features and Target

In [4]:
# Drop rows with missing target values
# The target column from 02_data_understanding.ipynb is 'price_1w_ahead'
df_clean = df_model.dropna(subset=['price_1w_ahead']).copy()

print(f"Rows after removing missing targets: {len(df_clean)}")

# Define target and features
target_col = 'price_1w_ahead'
y = df_clean[target_col]

# ====================================================================
# CRITICAL FIX: Exclude ALL non-lagged features to prevent data leakage!
# We must only use features that contain '_lag_' in their name.
# ====================================================================

# 1. Identify all features that are properly lagged
feature_cols = [col for col in df_clean.columns if '_lag_' in col]

# 2. Identify all columns that will be excluded
exclude_cols = [col for col in df_clean.columns if col not in feature_cols]

X = df_clean[feature_cols]

print(f"\n{'='*70}")
print("FEATURE SELECTION (DATA LEAKAGE PREVENTION)")
print(f"{'='*70}")
print(f"Target variable: {target_col}")
print(f"Number of features (ONLY lagged): {len(feature_cols)}")
print(f"Sample features: {feature_cols[:5]}...")
print(f"\nNumber of EXCLUDED columns (non-lagged): {len(exclude_cols)}")
print(f"Sample excluded: {exclude_cols[:5]}...")
print(f"\n✓ Excluding 'Europe_Base_Price' and all other non-lagged features.")
print(f"\nTarget statistics:")
print(y.describe())

Rows after removing missing targets: 384

FEATURE SELECTION (DATA LEAKAGE PREVENTION)
Target variable: price_1w_ahead
Number of features (ONLY lagged): 217
Sample features: ['price_lag_1w', 'price_lag_2w', 'price_lag_4w', 'egypt_disruption_lag_1w', 'europe_disruption_lag_1w']...

Number of EXCLUDED columns (non-lagged): 71
Sample excluded: ['SCFI_Index', 'Europe_Base_Price', 'global_total_events', 'global_disruption_events', 'extreme_crisis_events']...

✓ Excluding 'Europe_Base_Price' and all other non-lagged features.

Target statistics:
count     384.000000
mean     2485.953125
std      2259.645574
min       562.000000
25%       831.000000
50%      1132.000000
75%      3876.250000
max      7797.000000
Name: price_1w_ahead, dtype: float64


## Step 3: Train-Test Split (Time-Based)

In [5]:
# Time-based split (80/20)
split_idx = int(len(df_clean) * 0.8)

X_train = X.iloc[:split_idx]
X_test = X.iloc[split_idx:]
y_train = y.iloc[:split_idx]
y_test = y.iloc[split_idx:]

print(f"Training set: {len(X_train)} samples ({X_train.index.min().date()} to {X_train.index.max().date()})")
print(f"Test set: {len(X_test)} samples ({X_test.index.min().date()} to {X_test.index.max().date()})")
print(f"\nTraining target range: ${y_train.min():.2f} to ${y_train.max():.2f}")
print(f"Test target range: ${y_test.min():.2f} to ${y_test.max():.2f}")

Training set: 307 samples (2018-01-05 to 2024-01-26)
Test set: 77 samples (2024-02-02 to 2025-08-15)

Training target range: $562.00 to $7797.00
Test target range: $1154.00 to $5051.00


## Step 4: Feature Scaling

In [6]:
# Check for non-numeric columns before scaling
print("Checking data types in features...")
print(X_train.dtypes.value_counts())
print("\nNon-numeric columns:")
non_numeric_cols = X_train.select_dtypes(exclude=[np.number]).columns.tolist()
print(non_numeric_cols)

# Remove non-numeric columns from features
if len(non_numeric_cols) > 0:
    print(f"\nRemoving {len(non_numeric_cols)} non-numeric columns: {non_numeric_cols}")
    X_train = X_train.select_dtypes(include=[np.number])
    X_test = X_test.select_dtypes(include=[np.number])
    feature_cols = X_train.columns.tolist()
    print(f"\nRemaining features: {len(feature_cols)}")

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrames
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("\nFeatures scaled successfully!")
print(f"Scaled feature shape: {X_train_scaled.shape}")

Checking data types in features...
float64    217
Name: count, dtype: int64

Non-numeric columns:
[]



Features scaled successfully!
Scaled feature shape: (307, 217)


## Step 5: Define Evaluation Metrics

In [7]:
def evaluate_model(y_true, y_pred, model_name):
    """
    Calculate and display model performance metrics.
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r2 = r2_score(y_true, y_pred)
    
    metrics = {
        'Model': model_name,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape,
        'R²': r2
    }
    
    print(f"\n{'='*60}")
    print(f"{model_name} Performance")
    print(f"{'='*60}")
    print(f"RMSE: ${rmse:.2f}")
    print(f"MAE:  ${mae:.2f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"R²:   {r2:.4f}")
    print(f"{'='*60}")
    
    return metrics

# Store all model results
results = []

print("Evaluation functions defined!")

Evaluation functions defined!


## Step 6: Baseline Models

In [8]:
# ====================================================================
# BASELINE MODELS - Using Lagged Price Features (No Data Leakage!)
# ====================================================================

# Check what lagged price features are available
print("Available lagged price columns:")
price_lag_cols = [col for col in df_clean.columns if 'price_lag' in col.lower()]
print(price_lag_cols)

# Baseline 1: Naive Forecast (use price from 1 week ago)
if 'price_lag_1w' in df_clean.columns:
    print(f"\n✓ Using 'price_lag_1w' for naive baseline")
    y_pred_naive = df_clean.loc[y_test.index, 'price_lag_1w']
    metrics_naive = evaluate_model(y_test, y_pred_naive, 'Naive (Last Week)')
    results.append(metrics_naive)
    
    # Baseline 2: Moving Average (4-week average of LAGGED prices)
    print("\n✓ Calculating 4-week moving average from lagged prices")
    y_pred_ma = []
    
    for test_idx in y_test.index:
        # Use the lagged prices available at prediction time
        # We can use price_lag_1w, price_lag_2w, price_lag_3w, price_lag_4w
        lagged_prices = []
        for lag in [1, 2, 3, 4]:
            lag_col = f'price_lag_{lag}w'
            if lag_col in df_clean.columns and test_idx in df_clean.index:
                lagged_prices.append(df_clean.loc[test_idx, lag_col])
        
        if len(lagged_prices) >= 2:  # Need at least 2 lags
            y_pred_ma.append(np.mean(lagged_prices))
        elif 'price_lag_1w' in df_clean.columns:
            # Fallback to naive if not enough lags
            y_pred_ma.append(df_clean.loc[test_idx, 'price_lag_1w'])
        else:
            y_pred_ma.append(np.nan)
    
    y_pred_ma = pd.Series(y_pred_ma, index=y_test.index)
    
    # Remove any NaN predictions
    valid_idx = ~y_pred_ma.isna()
    if valid_idx.sum() > 0:
        metrics_ma = evaluate_model(y_test[valid_idx], y_pred_ma[valid_idx], '4-Week Moving Average')
        results.append(metrics_ma)
    else:
        print("⚠️  Could not create moving average baseline - insufficient lagged data")
else:
    print("\n❌ ERROR: price_lag_1w not found!")
    print("   This is created in 02_data_understanding.ipynb")
    print("   Please re-run that notebook to create lagged price features")
    print(f"\n   Available columns: {[c for c in df_clean.columns if 'price' in c.lower()]}")


Available lagged price columns:
['price_lag_1w', 'price_lag_2w', 'price_lag_4w', 'price_lag_3w', 'price_lag_5w', 'price_lag_6w', 'price_lag_7w', 'price_lag_8w', 'price_lag_1w_roll_mean_4w', 'price_lag_1w_roll_std_4w', 'price_lag_1w_roll_min_4w', 'price_lag_1w_roll_max_4w', 'price_lag_1w_cv_4w', 'price_lag_1w_roll_mean_8w', 'price_lag_1w_roll_std_8w', 'price_lag_1w_roll_min_8w', 'price_lag_1w_roll_max_8w', 'price_lag_1w_cv_8w', 'price_lag_1w_pct_change_1w', 'price_lag_1w_pct_change_4w']

✓ Using 'price_lag_1w' for naive baseline

Naive (Last Week) Performance
RMSE: $389.07
MAE:  $296.49
MAPE: 11.97%
R²:   0.8736

✓ Calculating 4-week moving average from lagged prices

4-Week Moving Average Performance
RMSE: $609.81
MAE:  $477.09
MAPE: 19.23%
R²:   0.6895


## Step 7: Linear Regression Models

In [9]:
# Linear Regression
print("\nTraining Linear Regression...")
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
y_pred_lr = lr_model.predict(X_test_scaled)
metrics_lr = evaluate_model(y_test, y_pred_lr, 'Linear Regression')
results.append(metrics_lr)

# Ridge Regression
print("\nTraining Ridge Regression...")
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train_scaled, y_train)
y_pred_ridge = ridge_model.predict(X_test_scaled)
metrics_ridge = evaluate_model(y_test, y_pred_ridge, 'Ridge Regression')
results.append(metrics_ridge)

# Lasso Regression
print("\nTraining Lasso Regression...")
lasso_model = Lasso(alpha=1.0, max_iter=5000)
lasso_model.fit(X_train_scaled, y_train)
y_pred_lasso = lasso_model.predict(X_test_scaled)
metrics_lasso = evaluate_model(y_test, y_pred_lasso, 'Lasso Regression')
results.append(metrics_lasso)


Training Linear Regression...

Linear Regression Performance
RMSE: $869.13
MAE:  $787.57
MAPE: 34.37%
R²:   0.3694

Training Ridge Regression...

Ridge Regression Performance
RMSE: $1263.33
MAE:  $1193.67
MAPE: 52.82%
R²:   -0.3324

Training Lasso Regression...



Lasso Regression Performance
RMSE: $902.16
MAE:  $837.50
MAPE: 38.06%
R²:   0.3205


## Step 8: Tree-Based Models

In [10]:
# Random Forest
print("\nTraining Random Forest...")
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
metrics_rf = evaluate_model(y_test, y_pred_rf, 'Random Forest')
results.append(metrics_rf)

# Gradient Boosting
print("\nTraining Gradient Boosting...")
gb_model = GradientBoostingRegressor(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)
metrics_gb = evaluate_model(y_test, y_pred_gb, 'Gradient Boosting')
results.append(metrics_gb)

# XGBoost
print("\nTraining XGBoost...")
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
metrics_xgb = evaluate_model(y_test, y_pred_xgb, 'XGBoost')
results.append(metrics_xgb)


Training Random Forest...



Random Forest Performance
RMSE: $435.00
MAE:  $347.25
MAPE: 14.77%
R²:   0.8420

Training Gradient Boosting...



Gradient Boosting Performance
RMSE: $419.49
MAE:  $309.18
MAPE: 12.49%
R²:   0.8531

Training XGBoost...



XGBoost Performance
RMSE: $458.53
MAE:  $358.67
MAPE: 14.90%
R²:   0.8245


## Step 9: Model Comparison

In [11]:
# Create comparison dataframe
df_results = pd.DataFrame(results)
df_results = df_results.sort_values('RMSE')

print("\n" + "="*80)
print("MODEL PERFORMANCE COMPARISON")
print("="*80)
print(df_results.to_string(index=False))
print("="*80)

# Visualize model comparison
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('RMSE', 'MAE', 'MAPE (%)', 'R²')
)

fig.add_trace(go.Bar(x=df_results['Model'], y=df_results['RMSE'], name='RMSE'), row=1, col=1)
fig.add_trace(go.Bar(x=df_results['Model'], y=df_results['MAE'], name='MAE'), row=1, col=2)
fig.add_trace(go.Bar(x=df_results['Model'], y=df_results['MAPE'], name='MAPE'), row=2, col=1)
fig.add_trace(go.Bar(x=df_results['Model'], y=df_results['R²'], name='R²'), row=2, col=2)

fig.update_xaxes(tickangle=45)
fig.update_layout(height=800, showlegend=False, title_text="Model Performance Metrics")
fig.show()


MODEL PERFORMANCE COMPARISON
                Model        RMSE         MAE      MAPE        R²
    Naive (Last Week)  389.071722  296.493506 11.971523  0.873624
    Gradient Boosting  419.492511  309.183599 12.491304  0.853089
        Random Forest  435.002535  347.247763 14.768081  0.842025
              XGBoost  458.534743  358.668755 14.897013  0.824471
4-Week Moving Average  609.812614  477.090909 19.226378  0.689545
    Linear Regression  869.129543  787.570133 34.374333  0.369370
     Lasso Regression  902.155801  837.498655 38.056769  0.320533
     Ridge Regression 1263.334740 1193.666496 52.822455 -0.332423


## Step 10: Feature Importance Analysis

In [12]:
# Get feature importances from XGBoost
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\nTop 20 Most Important Features (XGBoost):")
print(feature_importance.head(20).to_string(index=False))

# Visualize top 20 features
fig = px.bar(
    feature_importance.head(20),
    x='Importance',
    y='Feature',
    orientation='h',
    title='Top 20 Feature Importances (XGBoost)'
)
fig.update_layout(height=600, yaxis={'categoryorder': 'total ascending'})
fig.show()


Top 20 Most Important Features (XGBoost):
                                             Feature  Importance
                                        price_lag_1w    0.620485
                                        price_lag_2w    0.186957
                            price_lag_1w_roll_max_4w    0.052817
                          price_lag_1w_pct_change_4w    0.029662
                           price_lag_1w_roll_mean_4w    0.028447
               choke_gibraltar_strait_n_total_lag_1w    0.006687
                   sh_export_container_lag_1w_lag_1w    0.004972
                 choke_suez_canal_n_container_lag_2w    0.004253
                            price_lag_1w_roll_min_4w    0.003380
                          price_lag_1w_pct_change_1w    0.002748
choke_taiwan_strait_capacity_container_lag_2w_lag_2w    0.002718
           choke_gibraltar_strait_n_container_lag_1w    0.002680
    choke_bab_el-mandeb_strait_n_total_lag_2w_lag_1w    0.002534
              choke_gibraltar_strait_capacity_l

## Step 11: Prediction Visualization

In [13]:
# Create prediction dataframe
pred_df = pd.DataFrame({
    'Date': y_test.index,
    'Actual': y_test.values,
    'Naive': y_pred_naive.values,
    'XGBoost': y_pred_xgb,
    'Random Forest': y_pred_rf,
    'Gradient Boosting': y_pred_gb
}).set_index('Date')

# Plot actual vs predicted
fig = go.Figure()

fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df['Actual'], mode='lines+markers', 
                         name='Actual', line=dict(color='black', width=2)))
fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df['XGBoost'], mode='lines+markers',
                         name='XGBoost', line=dict(dash='dash')))
fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df['Random Forest'], mode='lines+markers',
                         name='Random Forest', line=dict(dash='dot')))
fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df['Naive'], mode='lines',
                         name='Naive Baseline', line=dict(color='gray', dash='dash', width=1), opacity=0.5))

fig.update_layout(
    title='Actual vs Predicted Europe Base Port Prices (Test Set)',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    height=600,
    hovermode='x unified'
)
fig.show()

## Step 12: Hyperparameter Tuning

In [14]:
print("\nPerforming hyperparameter tuning for XGBoost...")

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'min_child_weight': [1, 3, 5]
}

# Use TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=3)

# Grid search
xgb_tuned = xgb.XGBRegressor(random_state=42, n_jobs=-1)
grid_search = GridSearchCV(
    xgb_tuned,
    param_grid,
    cv=tscv,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best CV RMSE: ${-grid_search.best_score_:.2f}")

# Evaluate tuned model
y_pred_tuned = grid_search.predict(X_test)
metrics_tuned = evaluate_model(y_test, y_pred_tuned, 'XGBoost (Tuned)')
results.append(metrics_tuned)


Performing hyperparameter tuning for XGBoost...
Fitting 3 folds for each of 36 candidates, totalling 108 fits



Best parameters: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 200}
Best CV RMSE: $1371.56

XGBoost (Tuned) Performance
RMSE: $445.18
MAE:  $348.55
MAPE: 13.97%
R²:   0.8345


In [15]:
print("\nPerforming a more robust hyperparameter tuning for XGBoost...")

# Define a wider parameter grid
param_grid_robust = {
    'n_estimators': [200, 300, 400],
    'max_depth': [7, 10, 12],
    'learning_rate': [0.1, 0.15],
    'min_child_weight': [1, 3],
    'reg_alpha': [0.1, 0.5],
    'reg_lambda': [0.1, 0.5]
}

# Select top 20 features
top_20_features = feature_importance.head(20)['Feature'].tolist()
X_train_top20 = X_train[top_20_features]
X_test_top20 = X_test[top_20_features]

# Grid search with the new robust parameters on top 20 features
xgb_tuned_robust = xgb.XGBRegressor(random_state=42, n_jobs=-1)
grid_search_robust = GridSearchCV(
    xgb_tuned_robust,
    param_grid_robust,
    cv=TimeSeriesSplit(n_splits=5),
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    refit=True
)

grid_search_robust.fit(X_train_top20, y_train)

print(f"\nBest robust parameters: {grid_search_robust.best_params_}")
print(f"Best robust CV RMSE: ${-grid_search_robust.best_score_:.2f}")

# Evaluate the robust tuned model
y_pred_tuned_robust = grid_search_robust.predict(X_test_top20)
metrics_tuned_robust = evaluate_model(y_test, y_pred_tuned_robust, 'XGBoost (Robust Tuned)')
results.append(metrics_tuned_robust)


Performing a more robust hyperparameter tuning for XGBoost...
Fitting 5 folds for each of 144 candidates, totalling 720 fits



Best robust parameters: {'learning_rate': 0.15, 'max_depth': 7, 'min_child_weight': 3, 'n_estimators': 200, 'reg_alpha': 0.1, 'reg_lambda': 0.5}
Best robust CV RMSE: $938.90

XGBoost (Robust Tuned) Performance
RMSE: $431.24
MAE:  $334.62
MAPE: 13.60%
R²:   0.8447


## Step 13: Save Best Model and Results

In [16]:
import pickle
import os

# Create models directory
os.makedirs('models', exist_ok=True)

# Save best model
best_model_final = grid_search.best_estimator_
with open('models/best_model_xgboost.pkl', 'wb') as f:
    pickle.dump(best_model_final, f)
print("✓ Best model saved")

# Save scaler
with open('models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("✓ Scaler saved")

# Save feature names
with open('models/feature_names.pkl', 'wb') as f:
    pickle.dump(feature_cols, f)
print("✓ Feature names saved")

# Save results
df_results_final = pd.DataFrame(results).sort_values('RMSE')
df_results_final.to_csv('data/processed/model_comparison_results.csv', index=False)
print("✓ Results saved")

# Save predictions
pred_final = pd.DataFrame({
    'Date': y_test.index,
    'Actual': y_test.values,
    'Predicted': y_pred_tuned,
    'Error': y_test.values - y_pred_tuned
})
pred_final.to_csv('data/processed/test_predictions.csv', index=False)
print("✓ Predictions saved")

print("\n" + "="*80)
print("MODEL TRAINING COMPLETE!")
print("="*80)
print(f"Best Model: XGBoost (Tuned)")
print(f"Test RMSE: ${metrics_tuned['RMSE']:.2f}")
print(f"Test MAE: ${metrics_tuned['MAE']:.2f}")
print(f"Test MAPE: {metrics_tuned['MAPE']:.2f}%")
print(f"Test R²: {metrics_tuned['R²']:.4f}")
print("="*80)

✓ Best model saved
✓ Scaler saved
✓ Feature names saved
✓ Results saved
✓ Predictions saved

MODEL TRAINING COMPLETE!
Best Model: XGBoost (Tuned)
Test RMSE: $445.18
Test MAE: $348.55
Test MAPE: 13.97%
Test R²: 0.8345


In [17]:
# Display final comparison of ALL models
print("\n" + "="*80)
print("FINAL MODEL PERFORMANCE COMPARISON (ALL MODELS)")
print("="*80)
df_all_results = pd.DataFrame(results).sort_values('RMSE')
print(df_all_results.to_string(index=False))
print("="*80)

# Highlight the best model
best_model_name = df_all_results.iloc[0]['Model']
best_rmse = df_all_results.iloc[0]['RMSE']
best_r2 = df_all_results.iloc[0]['R²']

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   RMSE: ${best_rmse:.2f}")
print(f"   R²: {best_r2:.4f}")

# Check if naive baseline is winning (this would be bad!)
if 'Naive' in best_model_name:
    print("\n⚠️  WARNING: Naive baseline is the best model!")
    print("   This suggests:")
    print("   1. Features may not be predictive")
    print("   2. Possible data leakage")
    print("   3. Need to review feature engineering in 02_data_understanding.ipynb")
else:
    print(f"\n✅ Good! Advanced models outperform the naive baseline.")
    
    # Calculate improvement over naive
    naive_rmse = df_all_results[df_all_results['Model'].str.contains('Naive', case=False)]['RMSE'].values
    if len(naive_rmse) > 0:
        improvement = ((naive_rmse[0] - best_rmse) / naive_rmse[0]) * 100
        print(f"   Improvement over naive: {improvement:.1f}% reduction in RMSE")


FINAL MODEL PERFORMANCE COMPARISON (ALL MODELS)
                 Model        RMSE         MAE      MAPE        R²
     Naive (Last Week)  389.071722  296.493506 11.971523  0.873624
     Gradient Boosting  419.492511  309.183599 12.491304  0.853089
XGBoost (Robust Tuned)  431.238542  334.624954 13.604358  0.844747
         Random Forest  435.002535  347.247763 14.768081  0.842025
       XGBoost (Tuned)  445.175587  348.552512 13.970113  0.834550
               XGBoost  458.534743  358.668755 14.897013  0.824471
 4-Week Moving Average  609.812614  477.090909 19.226378  0.689545
     Linear Regression  869.129543  787.570133 34.374333  0.369370
      Lasso Regression  902.155801  837.498655 38.056769  0.320533
      Ridge Regression 1263.334740 1193.666496 52.822455 -0.332423

🏆 BEST MODEL: Naive (Last Week)
   RMSE: $389.07
   R²: 0.8736

⚠️  WARNING: Naive baseline is the best model!
   This suggests:
   1. Features may not be predictive
   2. Possible data leakage
   3. Need to revie

## Final Model Comparison Summary

Let's review all model performances to identify the best model.